# Домашняя работа 1. Spark RDD API
Вохрамеев Михаил ИУ6-55Б



### Задание 1
1) Рассчитайте расстояние от заданной точки (lat=55.751244, lng=37.618423) до каждого заведения общепита из набора данных. Выведите первые 10.
2) Рассчитайте расстояние между всеми заведениями общепита из набора данных. Выведите первые 10.
3) Выведите топ-10 наиболее близких и наиболее отдаленных заведений.

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
import numpy as np

In [2]:
conf = (
    pyspark.SparkConf()
        .set("spark.executor.memory", "1g")
        .set("spark.executor.core", "2")
        .set("spark.driver.host", "127.0.0.1")
        .setMaster("local[2]")
)

In [3]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/09 19:20:55 WARN Utils: Your hostname, MacBook-Air-4.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.15 instead (on interface en0)
25/11/09 19:20:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/09 19:20:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
schema = StructType([StructField(name="Id", dataType=IntegerType(), nullable=False),
                     StructField("Name", StringType(), True),
                     StructField("global_id", IntegerType(), False),
                     StructField("IsNetObject", StringType(), True),
                     StructField("OperatingCompany", StringType(), True),
                     StructField("TypeObject", StringType(), True),
                     StructField("AdmArea", StringType(), True),
                     StructField("District", StringType(), True),
                     StructField("Address", StringType(), True),
                     StructField("PublicPhone", StringType(), True),
                     StructField("SeatsCount", IntegerType(), True),
                     StructField("SocialPrivileges", StringType(), True),
                     StructField("Longitude_WGS84", DoubleType(), True),
                     StructField("Latitude_WGS84", DoubleType(), True),
                     StructField("geoData", StringType(), True)
])


In [5]:
df = spark.read.load(path="places.csv", 
                          format="csv",
                          schema=schema,
                          header="false", 
                          inferSchema="false", sep=",", nullValue="null", mode="DROPMALFORMED")
df.show(5)

+------+--------------------+---------+-----------+----------------+----------+--------------------+---------------+--------------------+---------------+----------+----------------+------------------+------------------+--------------------+
|    Id|                Name|global_id|IsNetObject|OperatingCompany|TypeObject|             AdmArea|       District|             Address|    PublicPhone|SeatsCount|SocialPrivileges|   Longitude_WGS84|    Latitude_WGS84|             geoData|
+------+--------------------+---------+-----------+----------------+----------+--------------------+---------------+--------------------+---------------+----------+----------------+------------------+------------------+--------------------+
| 21830|         Шоколадница|637379839|         да|     Шоколадница|      кафе|Центральный админ...|Басманный район|город Москва, Бау...|(495) 916-57-62|        48|             нет|37.678681276899106| 55.77302899555391|{type=Point, coor...|
| 22366|               МУ-МУ|6373798

In [6]:
def calculate_distance_to_position(row):
    lon = row.Longitude_WGS84
    lat = row.Latitude_WGS84
    name = row.Name
    
    lon_given = 37.618423
    lat_given = 55.751244
    
    lon, lat, lon_given_rad, lat_given_rad = map(np.radians, [lon, lat, lon_given, lat_given])
    
    calculated_distance = 2 * 6371 * np.arcsin(np.sqrt(np.sin((lat_given_rad-lat)/2)**2 + np.cos(lat_given) * np.cos(lat) * np.sin((lon_given_rad-lon)/2)**2))
    
    return (name, calculated_distance)
    

##### 1) Расстояние от заданной точки (Кремля) до заведений (первые 10):

In [7]:
rdd1 = df.rdd
distance_rdd = rdd1.map(calculate_distance_to_position)
results = distance_rdd.take(10)

print("Расстояние от Кремля до заведений (первые 10):")
for name, distance in results:
    print(f"{name}: {distance:.3f} км")

Расстояние от Кремля до заведений (первые 10):
Шоколадница: 4.849 км
МУ-МУ: 4.788 км
КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА: 4.548 км
Дом 12: 3.258 км
Чито-Ра: 3.328 км
Бар- буфет «Николай»: 3.067 км
Флорентини: 3.132 км
Beer Gik: 1.641 км
Погребок: 1.641 км
Пробка Гриль: 1.733 км


In [8]:
def calculate_distance_between_points(pair):
    row1, row2 = pair
    
    name1, lon1, lat1 = row1['Name'], row1['Longitude_WGS84'], row1['Latitude_WGS84']
    name2, lon2, lat2 = row2['Name'], row2['Longitude_WGS84'], row2['Latitude_WGS84']
    
    lon1_r, lat1_r, lon2_r, lat2_r = map(np.radians, [lon1, lat1, lon2, lat2])
    
    calculated_distance = 2 * 6371 * np.arcsin(np.sqrt(np.sin((lat2_r-lat1_r)/2)**2 + np.cos(lat1_r) * np.cos(lat2_r) * np.sin((lon2_r-lon1_r)/2)**2))
    
    return (name1, name2, calculated_distance)
    

In [9]:
rdd2 = df.rdd
pairs_rdd = rdd2.cartesian(rdd2)

filtered_pairs_rdd = pairs_rdd.filter(lambda x: x[0]['Id'] < x[1]['Id'])

##### 2) Расстояния между всеми парами заведений (первые 10):

In [10]:
distances_rdd = filtered_pairs_rdd.map(calculate_distance_between_points)

results = distances_rdd.take(10)

print("Расстояние между парами заведений (первые 10):")
for name1, name2, distance in results:
    print(f"Между {name1} и {name2} {distance:.3f} км")

Расстояние между парами заведений (первые 10):
Между Шоколадница и МУ-МУ 0.161 км
Между Шоколадница и КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА 0.844 км
Между Шоколадница и Дом 12 1.580 км
Между Шоколадница и Чито-Ра 1.559 км
Между Шоколадница и Флорентини 1.594 км
Между Шоколадница и Beer Gik 3.025 км
Между Шоколадница и Погребок 3.025 км
Между Шоколадница и Пробка Гриль 2.937 км
Между Шоколадница и TEMPO DI PASTA 2.941 км
Между Шоколадница и Хлеб насущный 2.941 км


##### 3) Расстояния между всеми парами заведений (10 самых ближних и 10 самых дальних):

In [11]:
results_asc_ordered = distances_rdd.takeOrdered(num=10, key=lambda x: x[2])

print("Расстояние между парами заведений (самые близкие 10):")
for name1, name2, distance in results_asc_ordered:
    print(f"Между {name1} и {name2} {distance} км")

Расстояние между парами заведений (самые близкие 10):
Между Погребок и Beer Gik 0.0 км
Между TEMPO DI PASTA и Хлеб насущный 0.0 км
Между Beer Gik и Kozlovna 0.0 км
Между Погребок и Kozlovna 0.0 км
Между Beer Gik и Па-Паэлья 0.0 км
Между Погребок и Па-Паэлья 0.0 км
Между Глав Пив Маг и Beermood 0.0 км
Между Блэк милк и Дабл Би 0.0 км
Между Jeffreys coffee и Мегобари 0.0 км
Между Блэк милк и Коленки Пчелы 0.0 км


In [12]:
results_desc_ordered = distances_rdd.takeOrdered(num=10, key=lambda x: -x[2])

print("Расстояние между парами заведений (самые дальние 10):")
for name1, name2, distance in results_desc_ordered:
    print(f"Между {name1} и {name2} {distance:.3f} км")

Расстояние между парами заведений (самые дальние 10):
Между МНИТИ и Calabash Club 5.456 км
Между МНИТИ и Залечь на дно 5.456 км
Между МНИТИ и Политех 5.456 км
Между МНИТИ и Антикафе Checkpoint 5.456 км
Между Шоколадница и МНИТИ 5.402 км
Между МНИТИ и Мареа 5.398 км
Между МНИТИ и БИБЛИОТЕКА Shisha Lounge 5.395 км
Между МНИТИ и Му-Му 5.395 км
Между МНИТИ и Стейк Хаус «Бизон» 5.394 км
Между МНИТИ и Jimmy Poy 5.382 км
